In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
train = tfds.load("malaria", as_supervised=True, data_dir="data\\",split="train")

for i in train:
    # print(i)
    for j in i:
        # print(j)
        (x, y, z) = j.shape
        print(x)
        break
    break

Alright, we need first a feature extractor then a classifier

What are the classes that we are looking for? It's infected and non-infected. 

Our first attempt will be to go minimalistic: After all, it's most resource-efificent to go with the least complexity necessary.

This means:
* Input
* One convolution layer
* Pool
* Classifier (output)

One issue we are facing is that the dataset images are of very different sizes. This wil

In [ ]:
tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), groups=1, activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)

tf.keras.layers.MaxPool2D(
    pool_size=(2, 2), strides=None, padding='valid', data_format=None, **kwargs
)

tf.keras.layers.GlobalAveragePooling2D(
    data_format=None, **kwargs
)


In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer

class Model(Model): 
    
    def __init__(self):
        super(Model, self).__init__()
        # Define the three layers.
        self.hidden_layer_1 = tf.keras.layers.Dense(units=256,
                                               activation=tf.keras.activations.sigmoid
                                               )
        self.hidden_layer_2 = tf.keras.layers.Dense(units=256,
                                               activation=tf.keras.activations.sigmoid
                                               )
        self.output_layer = tf.keras.layers.Dense(units=10,
                                               activation=tf.keras.activations.softmax
                                               )
    @tf.function
    def call(self, x):
        # Define the forward step.
        x = self.hidden_layer_1(x)
        x = self.hidden_layer_2(x)
        x = self.output_layer(x)
        return x